In [1]:
!pip install -U transformers
!pip install -U accelerate
!pip install -U datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 49.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:
      Successfully uninstalled transformers-4.47.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.6/336.6 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.2.1
    Uninstalling accelerate-1.2.1:
      Successfully uninstalled accelerate-1.2.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.4 MB/s eta 0:00:00
  Attempting u

In [2]:
import pandas as pd
import json
import requests

train = pd.read_csv("https://raw.githubusercontent.com/laxmimerit/All-CSV-ML-Data-Files-Download/master/mit_restaurant_search_ner/train.bio", sep="\t", header=None)
train.head()

,0,1
0,B-Rating,2
1,I-Rating,start
2,O,restaurants
3,O,with
4,B-Amenity,inside


In [3]:
response = requests.get("https://raw.githubusercontent.com/laxmimerit/All-CSV-ML-Data-Files-Download/master/mit_restaurant_search_ner/train.bio")
response = response.text
response = response.splitlines()

In [4]:
response

['B-Rating\t2',
 'I-Rating\tstart',
 'O\trestaurants',
 'O\twith',
 'B-Amenity\tinside',
 'I-Amenity\tdining',
 '',
 'O\t34',
 '',
 'B-Rating\t5',
 'I-Rating\tstar',
 'O\tresturants',
 'B-Location\tin',
 'I-Location\tmy',
 'I-Location\ttown',
 '',
 'O\t98',
 'B-Restaurant_Name\thong',
 'I-Restaurant_Name\tkong',
 'O\trestaurant',
 'B-Price\treasonable',
 'O\tprices',
 '',
 'O\ta',
 'O\tgreat',
 'O\tlunch',
 'O\tspot',
 'O\tbut',
 'B-Hours\topen',
 'I-Hours\ttill',
 'I-Hours\t2',
 'I-Hours\ta',
 'I-Hours\tm',
 'B-Restaurant_Name\tpassims',
 'I-Restaurant_Name\tkitchen',
 '',
 'O\ta',
 'O\tplace',
 'O\tthat',
 'O\tserves',
 'B-Dish\tsoft',
 'I-Dish\tserve',
 'I-Dish\tice',
 'I-Dish\tcream',
 '',
 'O\ta',
 'O\trestaurant',
 'O\tthat',
 'O\tis',
 'B-Rating\tgood',
 'B-Amenity\tfor',
 'I-Amenity\tgroups',
 '',
 'O\ta',
 'B-Dish\tsalad',
 'O\twould',
 'O\tmake',
 'O\tmy',
 'O\tday',
 '',
 'O\ta',
 'B-Cuisine\tsmoothie',
 'O\twould',
 'O\thit',
 'O\tthe',
 'O\tspot',
 '',
 'O\ta',
 'B-Dish\ts

In [5]:
train_tokens = []
train_tags = []

temp_tokens = []
temp_tags = []
for line in response:
  if line != "":
      tag, token = line.strip().split("\t")
      temp_tags.append(tag)
      temp_tokens.append(token)
  else:
      train_tokens.append(temp_tokens)
      train_tags.append(temp_tags)

      temp_tokens, temp_tags = [], []

len(train_tokens), len(train_tags)

(7659, 7659)

In [6]:
response = requests.get("https://raw.githubusercontent.com/laxmimerit/All-CSV-ML-Data-Files-Download/master/mit_restaurant_search_ner/test.bio")
response = response.text
response = response.splitlines()

test_tokens = []
test_tags = []

temp_tokens = []
temp_tags = []
for line in response:
    if line != "":
        tag, token = line.strip().split("\t")
        temp_tags.append(tag)
        temp_tokens.append(token)
    else:
        test_tokens.append(temp_tokens)
        test_tags.append(temp_tags)

        temp_tokens, temp_tags = [], []

len(test_tokens), len(test_tags)

(1520, 1520)

## HuggingFace Dataset Preparation

In [7]:
from datasets import Dataset, DatasetDict

df = pd.DataFrame({'tokens': train_tokens, 'ner_tags_str': train_tags})
train = Dataset.from_pandas(df)

df = pd.DataFrame({'tokens': test_tokens, 'ner_tags_str': test_tags})
test = Dataset.from_pandas(df)

dataset = DatasetDict({'train': train, 'test': test, 'validation': test})
dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags_str'],
        num_rows: 7659
    })
    test: Dataset({
        features: ['tokens', 'ner_tags_str'],
        num_rows: 1520
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags_str'],
        num_rows: 1520
    })
})

In [8]:
dataset['train'][0]

{'tokens': ['2', 'start', 'restaurants', 'with', 'inside', 'dining'],
 'ner_tags_str': ['B-Rating', 'I-Rating', 'O', 'O', 'B-Amenity', 'I-Amenity']}

In [9]:
unique_tags = set()
for tag in dataset['train']['ner_tags_str']:
    unique_tags.update(tag)

unique_tags = list(set([x[2:] for x in list(unique_tags) if x!='O']))

tag2index = {"O": 0}
for i, tag in enumerate(unique_tags):
    tag2index[f'B-{tag}'] = len(tag2index)
    tag2index[f'I-{tag}'] = len(tag2index)

index2tag = {v:k for k,v in tag2index.items()}

In [16]:
dataset = dataset.map(lambda example: {"ner_tags": [tag2index[tag] for tag in example['ner_tags_str']]})

Map:   0%|          | 0/7659 [00:00<?, ? examples/s]

Map:   0%|          | 0/1520 [00:00<?, ? examples/s]

Map:   0%|          | 0/1520 [00:00<?, ? examples/s]

In [17]:
dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags_str', 'ner_tags'],
        num_rows: 7659
    })
    test: Dataset({
        features: ['tokens', 'ner_tags_str', 'ner_tags'],
        num_rows: 1520
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags_str', 'ner_tags'],
        num_rows: 1520
    })
})

## Model Building

In [18]:
from transformers import AutoTokenizer
model_ckpt = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

dataset['train'][2]

{'tokens': ['5', 'star', 'resturants', 'in', 'my', 'town'],
 'ner_tags_str': ['B-Rating',
  'I-Rating',
  'O',
  'B-Location',
  'I-Location',
  'I-Location'],
 'ner_tags': [11, 12, 0, 9, 10, 10]}

In [19]:
input = dataset['train'][2]['tokens']
output = tokenizer(input, is_split_into_words=True)
tokenizer.convert_ids_to_tokens(output.input_ids)

['[CLS]', '5', 'star', 'rest', '##ura', '##nts', 'in', 'my', 'town', '[SEP]']

In [20]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples['tokens'], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:
            # if id=-100 then loss is not calculated
            if word_idx is None:
                label_ids.append(-100)

            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])

            else:
                label_ids.append(-100)

            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs['labels'] = labels

    return tokenized_inputs

In [21]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/7659 [00:00<?, ? examples/s]

Map:   0%|          | 0/1520 [00:00<?, ? examples/s]

Map:   0%|          | 0/1520 [00:00<?, ? examples/s]

In [23]:
tokenized_dataset['train'][2], dataset['train'][2]

({'tokens': ['5', 'star', 'resturants', 'in', 'my', 'town'],
  'ner_tags_str': ['B-Rating',
   'I-Rating',
   'O',
   'B-Location',
   'I-Location',
   'I-Location'],
  'ner_tags': [11, 12, 0, 9, 10, 10],
  'input_ids': [101, 1019, 2732, 2717, 4648, 7666, 1999, 2026, 2237, 102],
  'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'labels': [-100, 11, 12, 0, -100, -100, 9, 10, 10, -100]},
 {'tokens': ['5', 'star', 'resturants', 'in', 'my', 'town'],
  'ner_tags_str': ['B-Rating',
   'I-Rating',
   'O',
   'B-Location',
   'I-Location',
   'I-Location'],
  'ner_tags': [11, 12, 0, 9, 10, 10]})

## Data Collation and Metrics

In [24]:
# !pip install seqeval
# !pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=35fc197e5f1900101fc7b633a5c0a13583b98c3d41a5c7685357d5f02d0a48f5
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.7 MB/s eta 0:00:00


In [25]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [26]:
import evaluate
import numpy as np

metric = evaluate.load('seqeval')
label_names = list(tag2index)

def compute_metrics(eval_preds):
    logits, labels = eval_preds

    predictions = np.argmax(logits, axis=-1)
    true_labels = [[label_names[l] for l in label if l!=-100] for label in labels]

    true_predictions = [[label_names[p] for p, l in zip(prediction, label) if l != -100]
                        for prediction, label in zip(predictions, labels)]

    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)

    return {
        "precision": all_metrics['overall_precision'],
        'recall': all_metrics['overall_recall'],
        'f1': all_metrics['overall_f1'],
        'accuracy': all_metrics['overall_accuracy'],
    }

## Model Training

In [27]:
from transformers import AutoModelForTokenClassification
from transformers import TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_ckpt,
                                                        id2label=index2tag,
                                                        label2id=tag2index)

args = TrainingArguments("finetuned-ner", evaluation_strategy='epoch',
                         save_strategy='epoch',
                         learning_rate=2e-5,
                         num_train_epochs=3,
                         weight_decay=0.01)

trainer = Trainer(model=model, args=args,
                  train_dataset=tokenized_dataset['train'],
                  eval_dataset=tokenized_dataset['validation'],
                  data_collator=data_collator,
                  compute_metrics=compute_metrics,
                  tokenizer=tokenizer)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-27-9145e3c0fa14>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, args=args,


In [28]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.645800,0.304463,0.741329,0.800635,0.769841,0.908759
2,0.249700,0.284838,0.762352,0.798413,0.779966,0.914374
3,0.203400,0.285497,0.765932,0.805079,0.785018,0.917602


TrainOutput(global_step=2874, training_loss=0.31701752537227623, metrics={'train_runtime': 212.9835, 'train_samples_per_second': 107.882, 'train_steps_per_second': 13.494, 'total_flos': 105239751014754.0, 'train_loss': 0.31701752537227623, 'epoch': 3.0})

In [29]:
trainer.save_model("ner_distilbert")

## Prediction and Load & Save Model

In [30]:
from transformers import pipeline

checkpoint = "ner_distilbert"
pipe = pipeline('token-classification',
                model=checkpoint,
                aggregation_strategy='simple')

Device set to use cuda:0


In [31]:
pipe("which restaurant serves the best shushi in new york?")

[{'entity_group': 'Rating',
  'score': 0.97644335,
  'word': 'best',
  'start': 28,
  'end': 32},
 {'entity_group': 'Dish',
  'score': 0.9210123,
  'word': 'shushi',
  'start': 33,
  'end': 39},
 {'entity_group': 'Location',
  'score': 0.867264,
  'word': 'new york',
  'start': 43,
  'end': 51}]

## Save Model from Colab to Local, and extract requirements.txt

In [33]:
import os
from google.colab import files

# Zip the 'ner_distilbert' directory
!zip -r ner_distilbert.zip ner_distilbert

# Download the zip file
files.download('ner_distilbert.zip')

  adding: ner_distilbert/ (stored 0%)
  adding: ner_distilbert/training_args.bin (deflated 51%)
  adding: ner_distilbert/config.json (deflated 59%)
  adding: ner_distilbert/tokenizer_config.json (deflated 75%)
  adding: ner_distilbert/vocab.txt (deflated 53%)
  adding: ner_distilbert/model.safetensors (deflated 8%)
  adding: ner_distilbert/special_tokens_map.json (deflated 42%)
  adding: ner_distilbert/tokenizer.json (deflated 71%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [34]:
!pip freeze > colab_requirements.txt